<a href="https://colab.research.google.com/github/thoma1996/deep_learning_image_classification/blob/master/Kopie_von_prototype_for_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()
import structures
import simulator

data = simulator.simulate_all(simulator.presets['set_A'])

Saving simulator.py to simulator (3).py
Saving structures.py to structures (2).py


100%|██████████| 400/400 [00:04<00:00, 84.96it/s]


In [ ]:
!pip install ase

     |████████████████████████████████| 2.2MB 5.3MB/s 


In [ ]:
#define the class

import torch
import torch.nn as nn 
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,10,3)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(10,16,3)
        self.fc1=nn.Linear(16*10*10, 120)
        self.fc2=nn.Linear(120, 84)
        self.fc3=nn.Linear(84, 2)
        
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x=x.view(-1,16*10*10)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        
        return x
        
net=Net()
# test_data = torch.zeros((1,1,48,48))
# net(test_data)

In [ ]:
# define optimizer and loss

import torch.optim as optim

loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)


In [ ]:
# data from set_A

import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor()])

all_images = np.load('../data/set_A/train_images.npy')[:, None]
all_labels = np.load('../data/set_A/train_labels.npy')

data = [(image, label) for image, label in zip(all_images, all_labels)]

trainloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=0)

all_images = np.load('../data/set_A/test_images.npy')[:, None]
all_labels = np.load('../data/set_A/test_labels.npy')

data = [(image, label) for image, label in zip(all_images, all_labels)]
testloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=0)

classes = ('pristine', 'defect')

In [ ]:
# training the nn
#import simulator

# if set_in()=="set_B":
#     preset_key="set_B"
#     a=0
#     print(preset_key)
# elif set_in()=="set_A":
#     preset_key="set_A"
#     a=1
#     print(preset_key)

EPOCHS=5
for epoch in range(EPOCHS):
    
    running_loss=0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels= data
        optimizer.zero_grad()
        outputs=net(inputs)
        #print(i)
        #print(outputs)
        #print(inputs)
        loss=loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss+= loss.item()
        if i%200==199:
            print("[%d,%5d] loss: %.3f" % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
print("Finished Training")

[1,  200] loss: 0.000
[1,  400] loss: 0.000
[1,  600] loss: 0.000
[1,  800] loss: 0.000
[1, 1000] loss: 0.000
[1, 1200] loss: 0.000
[1, 1400] loss: 0.000
[1, 1600] loss: 0.000
[1, 1800] loss: 0.000
[2,  200] loss: 0.000
[2,  400] loss: 0.000
[2,  600] loss: 0.000
[2,  800] loss: 0.000
[2, 1000] loss: 0.000
[2, 1200] loss: 0.000
[2, 1400] loss: 0.000
[2, 1600] loss: 0.000
[2, 1800] loss: 0.000
[3,  200] loss: 0.000
[3,  400] loss: 0.000
[3,  600] loss: 0.000
[3,  800] loss: 0.000
[3, 1000] loss: 0.000
[3, 1200] loss: 0.000
[3, 1400] loss: 0.000
[3, 1600] loss: 0.000
[3, 1800] loss: 0.000
[4,  200] loss: 0.000
[4,  400] loss: 0.000
[4,  600] loss: 0.000
[4,  800] loss: 0.000
[4, 1000] loss: 0.000
[4, 1200] loss: 0.000
[4, 1400] loss: 0.000
[4, 1600] loss: 0.000
[4, 1800] loss: 0.000
[5,  200] loss: 0.000
[5,  400] loss: 0.000
[5,  600] loss: 0.000
[5,  800] loss: 0.000
[5, 1000] loss: 0.000
[5, 1200] loss: 0.000
[5, 1400] loss: 0.000
[5, 1600] loss: 0.000
[5, 1800] loss: 0.000
Finished T

In [ ]:
# show the accuracy

correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs= net(images)
        _, predicted=torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
print("Accuracy of the network: %d %%" %(100*correct/total))

Accuracy of the network: 99 %


In [ ]:
# shows the accuracy of each class

class_correct=list(0. for i in range(2))
class_total=list(0. for i in range(2))
with torch.no_grad():
    for data in testloader:
        images, labels =data
        outputs= net(images)
        _, predicted=torch.max(outputs, 1)
        c= (predicted==labels).squeeze()
        for i in range(4):
            label=labels[i]
            class_correct[label]+= c[i].item()
            class_total[label]+=1

for i in range(2):
    print("Accuracy of %5s : %2d %%" %(classes[i], 100*class_correct[i]/class_total[i]))

Accuracy of pristine : 100 %
Accuracy of defect : 99 %
